# BB84

In [ ]:
'''
File containing the functions and libraries required for performing the bb84 protocol
'''

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from sys import getsizeof
from scipy.optimize import curve_fit
from contextlib import contextmanager
import sys

from qiskit import *
from qiskit.qasm2 import dumps
from qiskit_aer import Aer

import base64
import random

# import operation as op
from functools import reduce

####################################################################### In bb84_reservoir #######################################################################

### Imports :

# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np
# import time
# import os
# from sys import getsizeof
# from scipy.optimize import curve_fit
# from contextlib import contextmanager
# import sys


### Functions used : 

# Define the hyperbolic function
def hyperbolic_fit(x, a, b):
    return a / x + b


#from sys import getsizeof
def Size(var):
    print(f" Sys size : {getsizeof(var)}", end = ", ".rjust(8 - len(f"{getsizeof(var)}")))
    
    try:
        print(f" np size : {var.nbytes}", end = " ")
    except:
        print(" np size : NA", end = " ")


def test(vars, labels):
    '''
    Size and Type : Prints the label of the variable and the corresponding size(with overhead) and the numpy size(if applicable). Also shows the 
    datatype of the variable. All in a justified manner.

    Data Examples : Prints the first 10 elements of the very first dimension of an array/list (e.g. in case of a 3-D array, will print the 
    array[0, 0, :10] element). If the array is only 1-D, will print the first 10 elements. If it's a single variable, the value will be printed.
    Next to each array example, '*' will be printed. The number of '*' printed corresponding to an array shows its dimensions.
    '''
    max_len = len(max(labels, key = len))

    print("\nSize and Type :\n")
    for item, label in zip(vars, labels):
        print(f"{label} {':'.rjust(max_len + 2 - len(label))} ", end = " ") 
        Size(item), print("    ", type(item), end = " "), print("")

    print("\n\nData Examples :\n ")
    for item, label in zip(vars, labels):
        print(f"{label} {':'.rjust(max_len + 2 - len(label))} ", end = " ") 
        
        try :
            try :
                print(item[0, :10], "**")
            except :
                print(item[:10], "*")
        
        except :
            print(item)    


######################################################################## In Qiskit_rebuilt_4 ################################################################


# from qiskit import *
# from qiskit.qasm2 import dumps
# from qiskit_aer import Aer

# import base64
# import numpy as np
# import random


### Functions used

# from qiskit import *
# from qiskit.qasm2 import dumps
# import random
def NoisyChannel(qc1, qc2, qc1_name, errors, noise = 5e-4):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates), simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        will occur in qc2, and then initializes another circuit qc2 with the introduced noise.
    ''' 
    
    # Retrieve quantum state from qasm code of qc1
    qs = [dumps(qc1[i]).split('\n') for i in range(len(qc1))]
    
    # Process the code to get the instructions
    parsed_instructions = []
    for i, qasm_code in enumerate(qs):
        for line in qasm_code:
            line = line.strip()    # removing leading/trailing whitespace
            if line.startswith(('x', 'h', 'measure')):
                line = line.replace('0', str(i))
                parsed_instructions.append(line)
    
    # Apply parsed instructions to qc2
    for instruction in parsed_instructions:
        if instruction.startswith('x'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)
            
        elif instruction.startswith('h'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].h(0)
        
        elif instruction.startswith('measure'):
            continue    # exclude measuring
            
        else:
            print(f"Unable to parse instruction: {instruction}")
            raise Exception('Unable to parse instruction')
    
    # Introducing noise (taking input)
    for instruction in parsed_instructions:
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)     # Apply bit-flip error
            errors[0] += 1
            
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].z(0)     # Apply phase-flip error
            errors[1] += 1

    return errors


# import random
def generate_random_bits(num):
    """This function generates a random array of bits(0/1) of size = num"""
    # bits = np.array([random.randint(0, 1) for _ in range(num)])    # Randomly fills the array with 0/1

    bit_string = ""
    for _ in range(num):
        rand_bit = random.randint(0, 1)     # Flip Coin
        bit_string += str(rand_bit)
        
    return bit_string


# import random
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    
    bases_string = ""
    for _ in range(num_of_bases):
        randBasis = random.randint(0, 1)     # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string
	

# from qiskit import *
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)     # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit == "1" :
            qc.x(0)

        if basis == 'X' :
            qc.h(0)
            
        encoded_qubits.append(qc)
            
    return encoded_qubits


# from qiskit_aer import Aer
# from qiskit import *
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it.
        - qubits : a series of 1-qubit Quantum Circuit
        - bases : a string of random [X, Z] bases"""

    # bits = np.zeros(len(bases), dtype = int)    # The results of measurements
    bits = ""
        
    for idx, (qubit, basis) in enumerate(zip(qubits, bases)):

        if basis == "X" :
            qubit.h(0)
            
        qubit.measure(0, 0)
        
        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        transpiled_circuit = transpile(qubit, simulator)
        result = simulator.run(transpiled_circuit, shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get)     # Max doesn't matter for simulator since there is only one shot.

        bits += str(measured_bit)
        # bits[idx] = int(measured_bit)
        
    return bits


# import numpy as np
def array_to_string(array):
    result = np.array2string(
        array, 
        separator = "", 
        max_line_width = (len(array)+3))
    return result.strip('[').strip(']')


# import base64
def convert_to_octets(key):

    octets = []
    num_octets = len(key) // 8

    for i in range(num_octets):
        start = i * 8
        end = start + 8
        octet = key[start:end]
        octets.append(int(octet, 2))

    return bytearray(octets)


######################################################################## In Hamming ########################################################################

# Importing Qiskit
# from qiskit import *
# from qiskit.qasm2 import dumps
# from qiskit_aer import Aer

# import numpy as np
# import operation as op
# from functools import reduce
# import random

### Functions used

# from functools import reduce
def hamming(bits, order):
    '''
    Takes a string of bits to be corrected (bob bits). Bit-wise sums the indices of elements which are '1'. The 0th bit stores the parity
    of the entire block. The location of the error is returned. 
    If the location is not '0', the current 0th parity is matched with that of the parity obtained after flipping the bit at the location obtained. 
    If the parity matches, then the error is found and corrected. If the parity doesn't match then there are more than 1 error.

    If the location is '0', then no error is present.
    
    '''
    
    loc = reduce(lambda x, y : x^y, [i for i, bit in enumerate(bits) if bit == 1])    # x^y will apply xor to the binary rep of i -> index of 1s
    # loc = reduce(op.XOR, [i for i, bit in enumerate(bob_bits) if bit == 1])
    print(f"{loc = }")
    
    binary_rep = f"{bin_rep(loc, order)}"

    par = sum(bits[i] for i in range(0, len(bits)))%2    # Parity of the entire block. It should be 0-Even

    print(f"\n Hamming : ", end = " ")
    if loc != 0 :
        if par != 0 :
            err_count = 1
            print(f"Error found at location : {loc}")

        else :
            err_count = 2
            print("2 errors found")
            
    else : 
        err_count = 0
        print("No errors found")

    print(f" {err_count = }, {loc = }, {binary_rep = }")
    
    return err_count, loc, binary_rep


# import numpy as np
def Order(bits):
    try : return np.ceil(np.log2(len(bits))).astype(int)
    except : return np.ceil(np.log2(Unprocessed_key_len)).astype(int)
        

def bin_rep(loc, order):
    '''
    Takes a number(int) and order/precision(int) as an input, and returns the binary form with the requested precision.
    '''
    bin_loc = bin(loc)[2:]
    bin_rep = f"{'0'*(order - len(bin_loc))}{bin_loc}"
    
    return bin_rep


# import numpy as np
def parity(order):
    '''
    Takes in order(int) as a parameter. Returns 2 arrays : 
       - parity_bits : an array containing '0' and the powers of 2 till 2^(order-1)
       - bin_parity : an array of the binary representation of elements of parity_bits   
    '''
    PARITY_DICT = {0:0, **{2**i : 0 for i in range(order)}}    # Initializes the PARITY_DICT
    # parity_bits = np.array([0] + [2**i for i in range(order)]).astype(int)
    bin_parity = np.array([bin_rep(int(i), int(order)) for i in PARITY_DICT.keys()])

    return PARITY_DICT, bin_parity
    
    
# import numpy as np
def parity_locs(order):
    '''
    Takes in order(int) as a parameter. Returns an array :
        - parity_locs : A block(array reshaped as square matrix) with 1 at the locations of parity bits
    '''
    parity_locs = np.full(2**order, '-', dtype = object)
    PARITY_DICT = parity(order)[0]
    
    for loc in PARITY_DICT.keys() : parity_locs[loc] = '1'

    return parity_locs


# import numpy as np
def block(bits, order):
    dim = int(2**(order/2))
    # bits = bits.astype(int)
    # print(f" {len(bits) = } ")
    # print(f'{type(bits[0])= }')
    
    if len(bits) != 2**order : return f"key size (= {len(bits)}) not an exponent of 2 : {bits}"
        
    elif not order%2 : 
        try :
            return(f"block : \n {bits.reshape(dim, dim)} \n Shape of the block : {dim}*{dim}")
        except : 
            int_bits = np.array([int(bit) for bit in bits])
            return(f"block : \n {int_bits.reshape(dim, dim)} \n Shape of the block : {dim}*{dim}")

    else :
        return(f"bit string(Order is odd, can't project to a block) : \n")#{bits} \n Shape of the block : {bits.shape}")

# import numpy as np
def create_parity_block(bits, order, PARITY_DICT):
    '''
    A function to take a string of bits shared through QKD, and to morph them into a parity block with parity bits(unchecked) embedded
    '''
    block = np.zeros(2**order).astype('uint8')
    ### Encode alice_keys and get the PARITY_DICT before proceeding to bob_key
    
    j = 0 
    for i in range(2**order) : 
        if i in PARITY_DICT.keys() : block[i] = PARITY_DICT[i]
        elif j < len(bits) :    
            block[i] = bits[j]
            j += 1

    block, PARITY_DICT = encode_parity(block, order, PARITY_DICT)
    
    return block


# import numpy as np
def encode_parity(bits, order, PARITY_DICT):
    # order = np.ceil(np.log2(len(alice_bits)))
    
    sub_block = int(2**(order - 1))
    parity_of = np.zeros((len(PARITY_DICT), sub_block)).astype(int)   # An array to store the locations affecting the parity p
    
    for p in range(1, order+1) :    # checking for 1 at position p. eg : bin(45) = 101101
    
        bit_index = 2**(p-1)
        highlight = np.zeros(2**order).astype(int)                        # Highlights the locations affected by the current parity bit
        # print(f"bin rep of {bit_index = } : {bin_parity[p]}")
        Sum = 0
    
        for i in range(sub_block):                                         #  Order-1 = 5. range(5) = 0, 1, 2, 3, 4 => order-2
            bin_index = bin_rep(i, order-1)                                # Index(in binary formin binary form) for the data bits : 5 digits : 00010
            bin_index = bin_index[: order-p] + '1' + bin_index[order-p :]
            index = int(bin_index, base = 2)                                # Gives the index(int) of the elements to be considered for the current parity element
            
            parity_of[p, i] = index
            # highlight[index] = 1

            if bit_index != index :
                Sum = np.mod(Sum + bits[index], 2)

        # PARITY_DICT[bit_index] = np.mod( sum( bits[parity_of[p, i]] for i in range(sub_block) if bit_index != parity_of[p, i] ), 2 )

        PARITY_DICT[bit_index]= Sum
        bits[bit_index] = PARITY_DICT[bit_index]
            
        # print(highlight.reshape(dim, dim))
    
    PARITY_DICT[0] = sum( bits[i] for i in range(1, 2**order) )%2
    bits[0] = PARITY_DICT[0]
    # print(f"Parity locations : \n{parity_of[1:]}") 
    
    print("\n Hamming Results : ", hamming(bits, order))
    print(f" Uncorrected {block(bits, order)}")
    
    return bits, PARITY_DICT

# RA

In [ ]:
'''
File containing functions and libraries required for executing Resource allocation protocol.
'''

import numpy as np
import random
import math
import networkx as nx
import matplotlib.pyplot as plt
import os
import time

from contextlib import contextmanager
import sys, os
import tqdm
import warnings

############################################################### Generating_Network_Topology.ipynb ###############################################################
# import networkx as nx
# import matplotlib.pyplot as plt
# import numpy as np 


def create_topology(edges, save_fig = False, draw = True):
	'''
	for creating and printing a network topology with node and edge labels.
	Requires :
		edges(2d-list) : contains tuples of form (source node, destination node, weight)
	'''
	
	all_values = set([nodes for row in edges for nodes in row[:2]])    # A list of all the values/nodes in first 2 columns : s & d
	correction = -min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges : nodes start form 0
	
	edges = [(u + correction, v + correction, w) for u, v, w in edges]
	
	g = nx.Graph()
	numNodes = max(all_values) + correction
	
	nodes = np.arange(1, numNodes + 1)
	g.add_nodes_from(nodes)
	
	g.add_weighted_edges_from(edges)
	weights = {(item[0], item[1]): item[2] for item in edges}
	
	# Position of nodes
	pos = nx.spring_layout(g, seed = 5) 
	nx.draw_networkx_nodes(g, pos, node_size =200)
	nx.draw_networkx_edges(g, pos, edgelist = edges, width = 2) 
	nx.draw_networkx_labels(g, pos, font_size = 7)
	nx.draw_networkx_edge_labels(g, pos, weights)
	
	print("Number of Nodes: ", numNodes)
	print("Number of Links :", len(g.edges()))
	
	if save_fig == True:
		figname = str(numNodes) + "-nodes.png"
		plt.savefig(figname)
	
	#Configuraiton
	ax = plt.gca()
	ax.margins(0.08)
	plt.axis('off')
	plt.tight_layout()
	plt.show()
	
	return g


def create_bi_topology(edges, save_fig = False, draw = True):
	'''
	for creating and printing a BIDIRECTIONAL network topology with node and edge labels.
	Requires :
		edges(2d-list) : contains tuples of form (source node, destination node, weight)
	
	NOTE : Requires the edges to contain every unidirectional edge; i.e. 2 entries for a single bidirectional edge
	'''
	
	all_values = set([nodes for row in edges for nodes in row[:2]])    # A list of all the values/nodes in first 2 columns : s & d
	correction = -min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges : nodes start form 0
	
	edges = [(u + correction, v + correction, w) for u, v, w in edges]
	
	g = nx.DiGraph()
	numNodes = max(all_values) + correction
	
	nodes = np.arange(1, numNodes + 1)
	g.add_nodes_from(nodes)
	
	g.add_weighted_edges_from(edges)
	weights = {(item[0], item[1]): item[2] for item in edges}
	
	# Position of nodes
	pos = nx.spring_layout(g, seed = 5) 
	nx.draw_networkx_nodes(g, pos, node_size =200)
	nx.draw_networkx_edges(g, pos, edgelist = edges, width = 2) 
	nx.draw_networkx_labels(g, pos, font_size = 7)
	nx.draw_networkx_edge_labels(g, pos, weights)
	
	print("Number of Nodes: ", numNodes)
	print("Number of Links :", len(g.edges()))
	
	if save_fig == True:
		figname = str(numNodes) + "-nodes.png"
		plt.savefig(figname)
	
	#Configuraiton
	ax = plt.gca()
	ax.margins(0.08)
	plt.axis('off')
	plt.tight_layout()
	plt.show()
	
	return g


def k_sp(g, k = 10):
	"""Calculates k shortest paths for all node pairs in a graph using NetworkX.
	
	Args:
		g: The input graph.
		node_list: A list of nodes in the graph.
		k: The number of shortest paths to calculate.
	
	Returns:
		A dictionary of dictionaries mapping node pairs to path-cost pairs.
	"""
	# Nodes start from 0
	
	num_nodes = len(g)
	k_sp_dict = {}   # {(n1, n2) : {p1 : c1, ..., pk : ck}, ..., (n_k_1, n_k) : {p1 : c1, ..., pk : ck} } ; p_i = (l1, l2, ..., l_j)
	nodes = list(g.nodes)
	
	for i in nodes  :    # (i, )
		for j in nodes :    # (, j)
			if i == j:
				continue
				
			k_shortest_paths = list(nx.shortest_simple_paths(g, source = i, target = j, weight='weight'))
			path_costs = [sum(g[s][d]['weight'] for s, d in zip(path, path[1:])) for path in k_shortest_paths]
	
			k_sp_dict[(i, j)] = {tuple(path) : cost for path, cost in zip(k_shortest_paths, path_costs)}
	
	return k_sp_dict

###################################################################### Generating_CRs.ipynb ###################################################################


class CR:

	"""
	This class represents a Connection Request (CR) object. A CR can be in different states (allocated, blocked) and has various attributes 
	such as source node, destination node, security level, status, allocated resources, path, and index.
	
	**Attributes:**
		index (int): Unique identifier for the CR object.
		s (int): Source node of the CR.
		d (int): Destination node of the CR.
		sl (str): Security Level of the CR ("high", "medium", "low").
		tk (int): time slots required (currently set to 1).
		status (str): Current status of the CR ("allocated", "blocked", "initialized").
		text (str): Text associated with the CR.
		path (list): List of nodes representing the current path for the CR (obtained using k_sp or shortest_path).
		allocated_resources (list): List containing allocated resources (QSC, ts).
	
	**Methods:**
		__init__(self, index, s, d, sl, tk, status): Initializes the CR object.
		update_status(self, status, allocated_resources=None, path=None): Updates the CR's status, allocated resources, and path.
		display_info(self): Prints information about the CR object.
	
	**Class Methods:**
		def generate_crs(cls, numCR=1): Generates a list of CR objects with random attributes.
		def create_priority_queue(cls, queues='PQ'): Creates a priority queue based on security level from the list of CRs.
		def SRCR(cls, CRs): Calculates and prints the Success Rate of Connection Requests (SRCR).
		def NSP(cls, CRs, channel='total'): Calculates and prints the Network Security Provision (NSP).
		def display_all(cls, CRs): Prints detailed information for all CR objects in the list.
	"""
	
	#CRs = []
	sl_values = ["high", "medium", "low"]
	sfw = {"high" : 5, "medium" : 4, "low" : 3}    # Security factor weight
	
	def __init__(self, index, s, d, sl, tk):     # Constructor : Gets called as soon as an object is declared
		self.index = index
		
		self.s = s
		self.d = d
		
		self.sl = sl    # SL can be hign, mediuum, low
		self.tk = tk
		self.status = status    # Status can be Allocatted, blocked
		self.text = 'initialized'
	
		self.path = None
		self.allocated_resources = [None, None]    # [QSC, ts]
		
	
	def update_status(self, status, allocated_resources = [None, None], path = None) :    # A method to update the status
		"""
		Updates the CR object's status, allocated resources, and path.
		
		Args:
			status (str): The new status for the CR ("allocated", "blocked", etc.).
			allocated_resources (list, optional): List containing allocated resources (QSC, ts). Defaults to None.
			path (list, optional): List of nodes representing the allocated path. Defaults to None.
		"""
		self.status = status
	
		self.path = path
		self.allocated_resources = allocated_resources
	
	
	def display_info(self):
		print(f"CR: Index : {self.index}, Source Node ={self.s}, Destination Node={self.d}, Security Level={self.sl}, status={self.status}, allocated resources = {self.allocated_resources}, Path : {self.path}")
	
	###################################################################################################################################################
	
	@classmethod
	def generate_crs(cls, numCR = 1):    
		"""
		Generates a list of CR objects with random attributes (source, destination, security level, status) based on the specified number.
		
		Args:
			numCR (int, optional): The desired number of CR objects to generate. Defaults to 1.
		
		Returns:
			list: A list of CR objects.
		"""
	
		
		cls.CRs = []
	
		# Creating a list of uniformly sampled SLs
		n = math.floor(numCR/3)
		uniform_sample = n*cls.sl_values
	
		for i in range(numCR - 3*n):    # the range would be either 1 or 2. i can either be 0 or (0, 1)
			 uniform_sample.append(random.choice(cls.sl_values))
	
		# Initializing individual CRs
		for i in range(numCR):
			# Generating (source, destination) pair
			s = random.randint(1, numNodes)
			d = random.randint(1, numNodes)   
	
			while s == d : d = random.randint(1, numNodes)    # Checking if the source and destination nodes are same
	
			# Assigning sl            
			sl = random.choice(uniform_sample)    # Randomly selecting from a uniform sample
			uniform_sample.remove(sl)    # Removing the first(any) equivalent sl, to maintain the uniformity
	
			# currently only dealing with tk = 1.
			tk = 1
	
			cr = cls(i, s, d, sl, tk)
			cls.CRs.append(cr)
		##
		return cls.CRs
	
	@classmethod
	def create_priority_queue(cls, queues = 'PQ'):
		"""
		Creates a priority queue for the CR objects based on their security level (high, medium, low).
		
		Args:
			queues (str, optional): Specifies whether to return all queues or just the combined priority queue.
				- 'PQ' (default): Returns only the combined priority queue.
				- 'all': Returns all three queues (high, medium, low) along with the combined priority queue.
		
		Returns:
			list or tuple:
				- If queues='PQ': Returns a list containing the combined priority queue.
				- If queues='all': Returns a tuple containing four elements:
					- The combined priority queue (list).
					- The high-priority queue (list).
					- The medium-priority queue (list).
					- The low-priority queue (list).
		"""
		CR_1 = []
		CR_0 = []
		CR_minus1 = []
	
		for cr in cls.CRs :
			if cr.sl == "high": 
				CR_1.append(cr)
			elif cr.sl == "medium": 
				CR_0.append(cr)
			else: 
				CR_minus1.append(cr)
	
		PQ = CR_1 + CR_0 + CR_minus1    # Priority queue
	
		if queues == 'all': 
			return PQ, CR_1, CR_0, CR_minus1
		else: 
			return PQ
			
	
	@classmethod
	def SRCR(cls, CRs):    # Returns the SRCR
		"""
		Calculates and prints the Success Rate of Connection Requests (SRCR).
		
		Args:
			CRs (list): List of CR objects.
		
		Returns:
			float: The SRCR value.
		
		Prints:
			The SRCR value to the console.
		"""
	
		num_allocated_crs = sum(1 for cr in CRs if cr.status == "allocated" )
		
		SRCR = num_allocated_crs/len(CRs)
		print(f"The Success rate of connection requests(SRCR) is : {SRCR}")
		
		return SRCR
		
	
	@classmethod
	def NSP(cls, CRs, channel = 'total'):    # Returns the total NSP, if no 2nd argument given
		"""
		Calculates and prints the Network Security Performance (NSP) for each security level and the total NSP.
		
		Args:
			CRs (list): List of CR objects.
			channel (str, optional): Specifies whether to return all NSP values or just the total NSP.
				- 'total' (default): Returns and prints the total NSP.
				- 'all': Returns a list containing NSP values for each security level and prints the total NSP.
		
		Returns:
			float or list:
				- If channel='total': Returns the total NSP value.
				- If channel='all': Returns a list containing NSP values for each security level.
		
		Prints:
			The total NSP and/or individual NSP values for each security level to the console.
		"""
	
		nsp = [0, 0, 0, 0]
		for i, priority in enumerate(cls.sl_values) :
			nsp[i+1] = sum(cls.sfw[priority] for cr in CRs if cr.status == "allocated" and cr.sl == priority)
			nsp[0] += nsp[i+1]
	
		print(f"The NSP of the network is : {nsp[0]}")
	
		if channel == 'all':
			return nsp
		else :
			return nsp[0]
	
	
	@classmethod
	def display_all(cls, CRs):
		for cr in CRs:
			cr.display_info()



def plot_crs(*CRs) :
	"""
	Plots the distribution of security levels (SL) for varying numbers of Connection Requests (CRs).
	
	Generates a line plot showing the counts of high, medium, and low security level CRs
	across different numbers of total CRs.
	
	Args:
		*CRs: Variable length argument list of CR objects (not used in this function).
	
	Returns:
		None
	
	Side Effects:
		Generates a plot showing the distribution of security levels.
	"""
	high = []
	mid = []
	low = []
	numNodes = 14
	for X in range(25, 850, 25):
		CRs = CR.generate_crs(X) 
		counts = dict(Counter([CR.sl for CR in CRs]))
		high.append(counts["high"])
		mid.append(counts["medium"])
		low.append(counts["low"])
	
	plt.plot(range(25, 850, 25), high, label = "high")
	plt.plot(range(25, 850, 25), mid, label = "mid")
	plt.plot(range(25, 850, 25), low, label = "low")
	
	plt.scatter(range(25, 850, 25), high, label = "high")
	plt.scatter(range(25, 850, 25), mid, label = "mid")
	plt.scatter(range(25, 850, 25), low, label = "low")
	
	plt.legend(loc = 'best')
	plt.title("Distribution of SL levels for various number of CRs")
	plt.xlabel("Num of CRs")
	plt.ylabel("Counts of SL")

####################################################################### Links.ipynb #############################################################################

# The number of dictionaries in the class can be reduced by combining the values with different keys

#import numpy as np
class Links:
	"""
	Represents a network of links connecting nodes. Each link has specific resources (time slots)
	allocated for different security levels (SL) and traditional data channels (TDC).
	
	Attributes:
		channel_ts (dict): A dictionary mapping security level ("tdc", "high", "medium", "low") to the
							corresponding total number of time slots in the channel.
		n_ts (dict): A dictionary mapping security level ("tdc", "high", "medium", "low") to the
					 number of time slots in the corresponding channel.
		channel (dict): A dictionary mapping security level ("tdc", "high", "medium", "low") to the
						 channel index.
		priority (dict): A dictionary mapping channel index (0-3) to the corresponding security level.
		total_ts (int): The total number of time slots in each link (sum of all security level time slots).
		links (np.ndarray, optional): A 2D NumPy array containing Link objects at corresponding node indices
									   (initialized in `initialize_links`).
		indices (np.ndarray, optional): A mask indicating the location of each link in the `links` array
										(initialized in `initialize_links`).
		ordered_indices (list, optional): A list of tuples representing ordered pairs of nodes for each link
										  (initialized in `initialize_links`).
	
	"""
	
	
	## Class variables
	
	#links = []
	#indices : a mask
	#unordered_indices = []    # Not useful in case of bidirectional graph
	#numLinks = len(ordered_indices)
	
	
	# Creating a dictionary with sl and the corresponding number of time-slots in the channel
	num_ts = {"tdc" : 47, "high" : 8, "medium" : 10, "low" : 12,
			  0 : 47, 1 : 8, 2 : 10, 3 : 12}
	link_ts = num_ts[1] + num_ts[2] + num_ts[3]    # Total ts in each link
	sl_channel = {"tdc" : 0, "high" : 1, "medium" : 2, "low" : 3, 
				  0 : "tdc", 1 : "high", 2 : "medium", 3 : "low"}
	
	# channel = {"tdc" : 0, "high" : 1, "medium" : 2, "low" : 3}    # A dictionary for channel from sl 
	# priority = {0 : "tdc", 1 : "high", 2 : "medium", 3 : "low"}    # A dictionary for sl from channel
	
	
	def __init__(self, nodes, weight):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
		self.nodes = nodes
		self.weight = weight
		
		self.lambda_tdc = np.ones(47, dtype = bool)
		
		#self.total_ts = Links.total_ts
		
		self.occupied_ts = np.zeros(4).astype(int)    # Stores the number of occupied time slots [total, q1, q2, q3]
		self.available_ts = np.array([Links.total_ts,    # Stores the number of available time slots [total, q1, q2, q3]
									 Links.num_ts[1], Links.num_ts[2], Links.num_ts[3]])
		
		self.lambda_q1 = np.ones(Links.num_ts[1], dtype = bool)    # for high sl
		self.lambda_q2 = np.ones(Links.num_ts[2], dtype = bool)    # for medium sl    
		self.lambda_q3 = np.ones(Links.num_ts[3], dtype = bool)    # for low sl
	
	
	def update_link(self, channel, slot):
		"""
		Updates the availability of a time slot in a specific channel.
	
		Args:
			channel (int): The channel index (0 for TDC, 1-3 for security levels).
			slot (int): The time slot index.
		"""
		#QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
		QSC = [1, 2, 3]
		TDC = [0]
		AC = QSC + TDC
		#if channel not in AC or ts < 0:
			#print("Invalid Update request!")
	
		if channel == 0:
			self.lambda_tdc[slot] = False
		
		elif channel == 1 and slot < len(self.lambda_q1):
			self.lambda_q1[slot] = False 
			
		elif channel == 2 and slot < len(self.lambda_q2):
			self.lambda_q2[slot] = False
			
		elif channel == 3 and slot < len(self.lambda_q3):
			self.lambda_q3[slot] = False
	
		else:
			raise ValueError("Invalid time slot value")
	
		if channel != 0:
			self.occupied_ts[channel] += 1 
			self.occupied_ts[0] += 1
	
			self.available_ts[channel] -= 1 
			self.available_ts[0] -= 1
		
	
	def display_info(self, wl_info = False):
		"""
		Displays information about the link, including available and occupied time slots.
	
		Args:
			wl_info (bool, optional): If True, also displays detailed wavelength information. Defaults to False.
		"""
		q1_count = self.available_ts[1] # nonzero => Available slots
		q2_count = self.available_ts[2] #== True
		q3_count = self.available_ts[3] #== True
		
		tdc_count = np.count_nonzero(self.lambda_tdc) #== True
		print(f"Link {self.nodes} : lambda_tdc_count = {tdc_count}, lambda_q1_count = {q1_count}, lambda_q2_count = {q2_count}, lambda_q3_count = {q3_count}, occupied_ts = {self.occupied_ts}, available_ts = {self.available_ts}")
	
		if wl_info:    # To show the wavelength occupancy
			print(f"QSC: \n q1 : {self.lambda_q1}, q2 : {self.lambda_q2}, q3 : {self.lambda_q3}, tdc : {self.lambda_tdc}")
		
	###################################################################################################################################################
	
	@classmethod
	def initialize_links(cls, edges):    # Initializing all the links and the individual link resources
		"""
		Initializes all links in the network based on the provided edges data.
	
		Args:
			edges (list): A list of tuples representing edges in the graph (source, destination, weight).
	
		Returns:
			tuple: A tuple containing the `links`, `indices`, and `ordered_indices` arrays.
		"""
		num_Nodes = max(set([nodes for row in edges for nodes in row[:2]]))    # A list of all the values/nodes in first 2 columns : s & d
		cls.total_ts = cls.link_ts * num_Nodes
		cls.links = np.zeros([num_Nodes+1, num_Nodes+1], dtype = object)    # Matrix containing link object at position s, d. Will have redundant entries.
	
		cls.ordered_indices = []
		for (s, d, w) in edges:
			nodes = (s, d)
			link = cls(nodes, w)
			cls.links[s, d] = link   
			cls.ordered_indices.append(nodes)
		# In the above call of __init__ constructor, the wavelength resources have also been initialized to all available(True)
					
		cls.indices = cls.links != 0
				
	
	@classmethod
	def path_resources(cls, path, sl):    # To check for the available time slots, following the continuity constraint
		"""
		Checks for available time slots along a specified path for a given security level.
	
		Args:
			path (list): A list of nodes representing the path.
			sl (str): The security level.
	
		Returns:
			tuple: A tuple containing lists of available time slots for TDC and the security level.
		"""
		available_tdcs = [True] * cls.num_ts["tdc"]    # For traditional data channel slots
		available_ts = [True] * cls.num_ts[sl]    # Creating a base boolean array of the size corresponding to the specific CR's sl
	
		for s, d in zip(path, path[1:]):    # Taking consecutive pairs of nodes and selecting the particular channel
			if sl == "high":
				band = cls.links[s, d].lambda_q1
			elif sl == "medium":
				band = cls.links[s, d].lambda_q2
			else:
				band = cls.links[s, d].lambda_q3
	
			# Checking for continuity constraints in ts of quantum channel
			available_ts = [a and b for a, b in zip(available_ts, band)]
			available_tdcs = [a and b for a, b in zip(available_tdcs, cls.links[s, d].lambda_tdc)]
			
		return available_ts, available_tdcs
	
	
	@classmethod
	def ASLC(cls, ch, path, aslc, beta_1 = 1, beta_2 = 0.5):
		"""
		Implements the Adaptive Spectrum Leasing Channel (ASLC) algorithm.
	
		Args:
			ch (int): The channel index.
			path (list): The path to be considered.
			aslc (str): The ASLC strategy (ASSL or AWSL).
			beta_1 (float, optional): The first threshold parameter for AWSL. Defaults to 1.
			beta_2 (float, optional): The second threshold parameter for ASSL. Defaults to 0.5.
	
		Returns:
			str: The selected security level.
		"""
		
		NT = []    # equivalent to dict channel_ts. starts from index 0 : for high priority
		OT = []    # Number of ts occupied along the ENTIRE path
		for i in range(1, 4):
			nt = cls.channel_ts[i]    # Total time slot in channel/n-th wavelength
			NT.append(nt)
				
			available_ts, available_tdcs = cls.path_resources(path, cls.priority[i])    # Available ts in the path for a particular wavelength
			ot = nt - np.count_nonzero(available_ts)    # Denotes the number of time-slot continuity occupied along the path for the given sl
			OT.append(ot)
				
		# ch could be 1, 2 or 3
		# n = ch - 1    # n could be 0, 1 or 2    # no need for n
	
		if aslc == "ASSL" and ch != 1:    # ch = 2 or 3. n = 1 or 2
			if OT[ch-1] >= beta_2 * NT[ch-1]:    # If the occupied resources in the higher priority are greater than a threshold, don't allocate to it
				QW = cls.priority[ch]
			else:
				QW = cls.priority[ch-1]
	
		elif aslc == "AWSL" and ch != 3:    # ch = 1 or 2. n = 0 or 1
			if OT[ch] >= beta_1 * NT[ch]:    # If the resources in current priority are more than a threshold, allocate to a lower priority
				QW = cls.priority[ch+1]
			else:
				QW = cls.priority[ch]
	
		else:    # For SSL and cases in ASSL but high priority, or AWSL but low priority
			QW = cls.priority[ch]
		 
		return QW
			
		
	@classmethod
	def FF(cls, path, cr, aslc = "ssl"):
		"""
		Performs First Fit (FF) spectrum allocation for a Connection Request (CR).
	
		Args:
			path (list): The path for the CR.
			cr (CR): The Connection Request object.
			aslc (str, optional): The ASLC strategy to use. Defaults to "ssl".
	
		Returns:
			bool: True if allocation is successful, False otherwise.
		"""
		# First check for availability of resources
		
		cr.sl = cls.ASLC(cls.channel[cr.sl], path, aslc)
		sl = cr.sl
		available_ts, available_tdcs = cls.path_resources(path, sl)    # List of available time slots in the channel along the entire path
	
		if cr.tk > len(available_ts):    # ts is 1 in our case by default. But in case we change it, this line will be useful
			raise ValueError("CR duration cannot be longer than the available time-slots in a channel")
			# Can also block it instead
		 
		if np.count_nonzero(available_ts) == 0 or np.count_nonzero(available_tdcs) == 0:    # A pre-condition to check if available slots are present
			cr.update_status("blocked")
			return False
	
		# Some issue here
		qscs = np.nonzero(available_ts)[0][0]    # This function will give the index of 1st non-zero/True element in the list
		tdcs = np.nonzero(available_tdcs)[0][0]    # At present, assuming only one tdcs is needed
		
		for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
			link = links[s, d]
	
			link.update_link(cls.channel["tdc"], tdcs)
			link.update_link(cls.channel[sl], qscs)    # Updating the i_th ts in channel for sl to False
	
		allocated_resources = [cls.channel[sl], qscs]
		cr.update_status("allocated", allocated_resources, path)
		#print(f"allocated ts_{qscs} successfully to CR {cr.index}")
		return True
		
	
	@classmethod
	def cipher(cls, cr) :
	
		path = cr.path
		
		for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
			link = links[s, d]
	
			key = link.bb84()    # Incomplete
			link.tdc(key, cr.text)    # Incomplete
			
			link.update_link(cls.channel["tdc"], tdcs)
			link.update_link(cls.channel[sl], qscs)    # Updating the i_th ts in channel for sl to False
	
		allocated_resources = [cls.channel[sl], qscs]
		cr.update_status("allocated", allocated_resources, path)
		
		return True
	
		
	@classmethod
	def TUR(cls, channel = 'total'):    # Returns the total TUR, if no 2nd argument given    
		"""
		Calculates the Timeslot Utilization Ratio (TUR) for the network.
	
		Args:
			channel (str, optional): Specifies whether to calculate TUR for all channels or a specific channel.
				- 'total' (default): Calculates the overall TUR.
				- Other values: Calculates TUR for the specified channel (not implemented).
	
		Returns:
			float or list:
				- If `channel` is 'total', returns the overall TUR.
				- Otherwise, returns a list of TUR values for each channel.
		"""
		# Only for quantum channel
		util_ts = [0, 0, 0, 0]
		tur = util_ts
		num_links = len(cls.ordered_indices)    # This also takes in consideration the symmetric part of a unidirectional link
		total_network_ts = num_links * cls.total_ts    # = 1260
		
		for nodes in cls.ordered_indices :
			link = cls.links[nodes]
			util_ts += link.occupied_ts
	
		tur[0] = util_ts[0]/total_network_ts
		for i in range(1, 4):
			tur[i] = util_ts[i]/(len(cls.ordered_indices)*cls.channel_ts[i])
		
		print("The time-slot utilization ratio(TUR) is : ", tur)
		
		if channel == 'all':
			return tur
	
		else: 
			return tur[0]       
			
	
	@classmethod
	def display_all_links(cls, wl_info = False):
		for nodes in cls.ordered_indices :
			cls.links[nodes].display_info(wl_info)


# Helper

In [ ]:
from contextlib import contextmanager
import sys, os
import warnings



####################################################################### Functions #######################################################################


@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        try:  
            sys.stdout = devnull
            sys.stderr = devnull        
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


####################################################################### Functions #######################################################################


@contextmanager
def suppress_std(target = ["out"]): 
    with open(os.devnull, "w") as devnull:
        
        if "out" in target: old_stdout = sys.stdout
        if "err" in target: old_stderr = sys.stderr
        
        try:  
            if "out" in target: sys.stdout = devnull
            if "err" in target: sys.stdout = devnull        
            yield
            
        finally:
            if "out" in target: sys.stdout = old_stdout
            if "err" in target: sys.stderr = old_stderr


####################################################################### Functions #######################################################################


def track(lim, string = 'Iteration'):
    print(f'{string}: ', end = '')
    for i in range(1, lim+1):
        ''' Any operation(s) requiring a loop '''
        # print(f"\r Iteration : [{'='*i}>{' '*(100-i)}] {i}/100", end = '')
        print('\r{}'.format(string), end = '')


####################################################################### Functions #######################################################################


# Suppress all warnings globally
warnings.filterwarnings("ignore")

@contextmanager
def force_suppress():
    """Suppress all output by redirecting stdout and stderr to os.devnull."""
    with open(os.devnull, 'w') as fnull:
        old_stdout = os.dup(1)
        old_stderr = os.dup(2)
        os.dup2(fnull.fileno(), 1)
        os.dup2(fnull.fileno(), 2)
        try:
            yield
        finally:
            os.dup2(old_stdout, 1)
            os.dup2(old_stderr, 2)
            os.close(old_stdout)
            os.close(old_stderr)
